# Airline Sentiment Classification Based on the Passengers Tweets.

First of All, Importing all libraries required for data analysis and data visualizations.

In [ ]:
import pandas as pandasInstance
import numpy as numpyInstance
import matplotlib.pyplot as matplotlibInstance
import seaborn as seabornInstance

Doing setting for inline data visualizations.

In [ ]:
%matplotlib inline

Now Importing the Data Related to Passangers Tweets.

In [ ]:
tweets_Data = pandasInstance.read_csv('../input/Tweets.csv')

Checking the Header of the Data Loaded.

In [ ]:
tweets_Data.head()

Now Checking the Information about all the Columns.

In [ ]:
tweets_Data.info()

Getting Some Basic Statistical Data of The Tweets Data.

In [ ]:
tweets_Data.describe()

## Part 1: Some Basic Data Exploration and Data Visualizations.

Counting the Number of Different Airlines, present in the data.

In [ ]:
matplotlibInstance.figure(figsize=(20,10))
matplotlibInstance.tight_layout()
seabornInstance.countplot(x='airline',data=tweets_Data)

##### From Above Count Plot We can Colclude that "United" Airline has Most Tweets.

Finding Out From Which Time Zone Most of The Passangers Have Made Tweets.

In [ ]:
groupedByTimeZone = tweets_Data.groupby('user_timezone')

In [ ]:
numberOfTweetsFromDifferentTiemZones = groupedByTimeZone['user_timezone'].count()

In [ ]:
topFiveTimeZonesWithMostTweets = numberOfTweetsFromDifferentTiemZones.sort_values(ascending=False).head(5)

In [ ]:
topFiveTimeZonesWithMostTweets

##### Most Tweets from Passangers are from Eastern Time, Central Time, Pacific Time, Quito and Atlantic Time.

Let's Check Where Most Of the Airline Sentiment Confidence Lies.

In [ ]:
tweets_Data['airline_sentiment_confidence'].hist(color='green',figsize=(20,10))

We Can See that Most Airline Sentiment Confidence is Between 0.7 to 1.0

Counting the Length of Each Tweet.

In [ ]:
tweets_Data['Tweet Length'] = tweets_Data['text'].apply(len)

Grabbing the Information of the Most Length Tweet.

In [ ]:
tweets_Data[tweets_Data['Tweet Length'] == 186]['text'].iloc[0]

In [ ]:
tweets_Data.hist(column='Tweet Length',by='airline_sentiment',figsize=(20,10),color='red',bins=100)

##### From Above Histograms We can Conclude that tweets with Postiive Aspect has More Words on average and Neutral Too.

## Part 2: Cleaning and Normalizing the Tweets Text.

Importing the NLTK for the Purpose of Removing the Stopping Words.

In [ ]:
import nltk
from nltk.corpus import stopwords

Importing the Strings Library in order to Remove the Punctuation Marks and Other Things.

In [ ]:
import string

Now Grabbing the All Tweet's Text.

In [ ]:
tweets_text = tweets_Data['text']

Removing the Punctuation Marks of the Text and also the Stopping words.

In [ ]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

Now applying the Above Function to the Tweets Text

In [ ]:
withoutStopWordsandPunctuations = tweets_Data['text'].apply(text_process)

Importing the CountVectorizer to Make the Bag of Words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(tweets_Data['text'])

In [ ]:
print(len(bow_transformer.vocabulary_))

In [ ]:
tweets_bow = bow_transformer.transform(tweets_Data['text'])

Looking at the Shape and Amount of Non Zero Occurances.

In [ ]:
print('Shape of Sparse Matrix: ', tweets_bow.shape)
print('Amount of Non-Zero occurences: ', tweets_bow.nnz)

Checking the Sparsity.

In [ ]:
sparsity = (100.0 * tweets_bow.nnz / (tweets_bow.shape[0] * tweets_bow.shape[1]))
print('sparsity: {}'.format((sparsity)))

## Part 3: Using the TF-IDF.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(tweets_bow)

Transfering the Bag of Words into the TF-IDF at once.

In [ ]:
tweets_tfidf = tfidf_transformer.transform(tweets_bow)
print(tweets_tfidf.shape)

## Part 4: Training and Splitting the Model.

For this purpose we will use the Naive Bays Classifier Algorithm.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(tweets_tfidf, tweets_Data['airline_sentiment'])

## Part 5: Evaluation of the Model.

In [ ]:
all_predictions = spam_detect_model.predict(tweets_tfidf)
print(all_predictions)

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(tweets_Data['airline_sentiment'], all_predictions))